<a href="https://colab.research.google.com/github/Peng-Lei/ipynb/blob/main/Embedding%E5%85%A5%E9%97%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#本文主要介绍如何搭建本地的Embedding模型

在大多数Blog以及向量DB的使用样例中，大家都是用OpenAIEmbeddings

虽然OpenAI Embeddings API提供了一种方便、高效的方式来生成文本表示，关键是它收费啊，而且依赖网络。所以构建一个本地的Embedding服务很有必要

#安装Embedding
通过pip安装依赖
InstructorEmbedding
sentence_transformers

In [ ]:
!pip install InstructorEmbedding
!pip install sentence_transformers

#使用hkunlp/instructor-large模型
从https://huggingface.co/spaces/mteb/leaderboard上看
instructor-large排名13，还可以关键是开源

In [ ]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
sentence = "I am Peng Lei"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction, sentence]])
print(embeddings)

这个仅仅是演示了通过本地的Embeddings模型进行向量化，并没有展示如何跟LangChain集成

#与LangChain集成
LangChain是一个优秀的大模型应用框架，但是基本上所有的Example都使用了OpenAIEmbeddings()

现在我们将OpenAIEmbeddings()替换成本地的Sentence Embedding

其实只需要一行代码修改：
embedding = OpenAIEmbeddings()
替换成
embedding = HuggingFaceInstructEmbeddings()

上面将数据Embedding后就可以调用DB Vector进行保存了

##替换Model
在LangChain替换模型是一个比较简单的操作
LangChain开发了HuggingFaceInstructEmbeddings接口用于加载模型
我们只需要更换其参数model_name即可

In [ ]:
!pip install langchain

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
embedding = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')

#哪个Model更适合中文环境
instructor-large中文表现不佳
那我们就直接找Chinese Sentence Embeddings Model

https://huggingface.co/shibing624/text2vec-base-chinese

#对比text2vec-base-chinese和text2vec-base-chinese

先进行数据加载

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("shibing624/nli_zh","STS-B")
for i in range(10):
    print(dataset['test'][i])

编写代码对比text2vec-base-chinese、instructor-large两个model在中文情况下的表现

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

In [ ]:
model1 = SentenceTransformer("shibing624/text2vec-base-chinese")
model2 = INSTRUCTOR("hkunlp/instructor-large")

for i in range(10):
  data = dataset['test'][i]
  print(data)
  sentence1 = data['sentence1']
  sentence2 = data['sentence2']
  print("text2vec-base-chinese 测试", util.cos_sim(model1.encode(sentence1), model1.encode(sentence2)))
  print("instructor-large 测试", util.cos_sim(model2.encode(sentence1), model2.encode(sentence2)))

#把model下载到本地

运行程序的时候，每次都远程下载Model，对于本地应用来说，是比较麻烦耗时的操作

最好是下载到本地，然后通过全路径加载
embedding = HuggingFaceEmbeddings(model_name='./huggingface/shibing624_text2vec-base-chinese')